# Orchestrating Jobs, Model Registration, Continuous Deployment, and Lineage Tracking with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Workflow service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

And the SageMaker Lineage service makes it easy to track all the artifacts created in a SageMaker Machine Learning Pipeline from start to finish.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Parametrized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

## SageMaker Model Registry

Amazon SageMaker Model Registry supports the following:

* Catalog models after the training step - data scientists run tens to thousands of experiments and may select a small set of models as candidates for production.
* Manage model versions - data scientists can register new models which will be automatically versioned in the model registry.
* Compare models - data scientists can run model evaluation steps in Tioga pipeline and generate model metrics (e.g. accuracy metrics and bias metrics) which are recorded in the Model Registry and can be used to compare model versions.
* Approve models - data scientists can mark model versions as “approved” or “rejected”. Alternately, the Tioga pipeline can also automate the model approvals. If there is a deployment pipeline associated with a Model and a live endpoint, then the model version is propagated in production. Currently SageMaker supports a Blue/ Green update, but as part of Yosemite, we are adding support for Canary and Rolling deployments also.
* Deploy models in different AWS accounts - models in the Model Registry support resource sharing across accounts which enables models built in data scientist accounts to be deployed in different pre-production and production accounts.

## SageMaker Lineage

Amazon SageMaker Lineage supports the following:

* Automatically tracks all the artifacts created in a machine learning workflow from start to finish.  Modeled as a directed graph like structure.
* Explore the lineage artifacts with easy to use SDK methods.


## Notebook Overview

This notebook shows how to:

### SageMaker Workflows

* Define a set of Workflow Parameters that can be used to parametrize a Workflow Pipeline
* Define a Processing step that performs cleaning and feature engineering, splitting the input data into train and test data sets
* Define a Training step that trains a model on the pre-processed train data set
* Define a Processing step that evaluates the trained model's performance on the test data set
* Define a Register Model step that creates a model package from the estimator and model artifacts used in training
* Define a Conditional step that measures a condition based on output from prior steps and conditionally executes the Register Model step
* Define and create a Pipeline in a Workflow DAG, with the defined parameters and steps defined
* Start a Pipeline execution and wait for execution to complete
* Download from S3 the model evaluation report for examination
* Start a second Pipeline execution

### SageMaker Model Registry

* Create a SageMaker Project based on the Model Package Group name from the pipeline execution defined before
* Observe CI/CD code pipeline on subsequent successful executions of the pipeline and the registration of a new Model Package version.

### SageMaker Lineage

Amazon SageMaker Lineage supports the following:

* Provide the inputs and outputs of SageMaker job artifacts

# A SageMaker Workflow

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and model registration:

![A typical ML Application pipeline](img/pipeline-full.png)

### Create SageMaker Clients and Session

First, we create a new SageMaker Session in the `us-east-2` region. We also acquire the role arn for the session.

This role arn should be the execution role arn that you set up in the Prerequisites section of this notebook.

In [1]:
# !pip install -q --upgrade pip

In [2]:
# !pip install -q sagemaker==2.23.1

In [3]:
# !pip install -q sagemaker-experiments==0.1.25

In [4]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [5]:
import time
timestamp = str(int(time.time() * 10**7))
print(timestamp)

16096406156145956


# Track Pipeline In `Experiment`

In [6]:
pipeline_name = 'BERT-pipeline-{}'.format(timestamp)

In [7]:
import time
from smexperiments.experiment import Experiment

timestamp = int(time.time())

experiment = Experiment.create(
                experiment_name=pipeline_name,
                description='Amazon Customer Reviews BERT Pipeline Experiment', 
                sagemaker_boto_client=sm)

experiment_name = experiment.experiment_name
print('Experiment name: {}'.format(experiment_name))

Experiment name: BERT-pipeline-16096406156145956


# Create the `Trial`

In [8]:
import time
from smexperiments.trial import Trial

timestamp = int(time.time())

trial = Trial.create(trial_name='trial-{}'.format(timestamp),
                     experiment_name=experiment_name,
                     sagemaker_boto_client=sm)

trial_name = trial.trial_name
print('Trial name: {}'.format(trial_name))

Trial name: trial-1609640615


# Create the `Experiment Config`'s for Each Step

In [9]:
experiment_config_prepare = {
    'ExperimentName': experiment_name,
    'TrialName': trial_name,
    'TrialComponentDisplayName': 'prepare'
}

In [10]:
experiment_config_train = {
    'ExperimentName': experiment_name,
    'TrialName': trial_name,
    'TrialComponentDisplayName': 'train'
}

In [11]:
experiment_config_evaluate = {
    'ExperimentName': experiment_name,
    'TrialName': trial_name,
    'TrialComponentDisplayName': 'evaluate'
}

In [12]:
experiment_config_model_register = {
    'ExperimentName': experiment_name,
    'TrialName': trial_name,
    'TrialComponentDisplayName': 'model_register'
}

# Specify the Raw Inputs S3 Location

In [13]:
raw_input_data_s3_uri = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(raw_input_data_s3_uri)

s3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/


In [14]:
!aws s3 ls $raw_input_data_s3_uri

2021-01-02 02:04:30   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2021-01-02 02:04:31   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz


# Setup Processing Job Hyper-Parameters

In [15]:
# max_seq_length=64
# train_split_percentage=0.90
# validation_split_percentage=0.05
# test_split_percentage=0.05
balance_dataset='True'
# processing_instance_count=1
# processing_instance_type='ml.c5.2xlarge'

# Define Parameters to Parametrize Pipeline Execution

We define Workflow Parameters by which we can parametrize our Pipeline and vary the values injected and used in Pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:

* `ParameterString` - representing a `str` Python type
* `ParameterInteger` - representing an `int` Python type
* `ParameterFloat` - representing a `float` Python type

These parameters support providing a default value, which can be overridden on pipeline execution. The default value specified should be an instance of the type of the parameter.

The parameters defined in this workflow below include:

* `processing_instance_type` - The `ml.*` instance type of the processing job.
* `processing_instance_count` - The instance count of the processing job. For illustrative purposes only: 1 is the only value that makes sense here.
* `training_instance_type` - The `ml.*` instance type of the training job.
* `model_approval_status` - What approval status to register the trained model with for CI/CD purposes. Defaults to "PendingManualApproval". (NOTE: not available in service yet)
* `input_data` - The URL location of the input data

## BERT Pipeline Parameters

In [16]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
#    ParameterBoolean
)

## General Parameters

In [17]:
exp_name = ParameterString(
    name="ExperimentName",
    default_value=experiment_name,
)

In [18]:
train_split_percentage = ParameterFloat(
    name="TrainSplitPercentage",
    default_value=0.90,
)

### Processing Step

In [19]:
input_data = ParameterString(
    name="InputData",
    default_value=raw_input_data_s3_uri,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)

processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.c5.2xlarge"
)

max_seq_length = ParameterInteger(
    name="MaxSeqLength",
    default_value=64,
)

balance_dataset = ParameterString(
    name="BalanceDataset",
    default_value="True",
)
    
train_split_percentage = ParameterFloat(
    name="TrainSplitPercentage",
    default_value=0.90,
)

validation_split_percentage = ParameterFloat(
    name="ValidationSplitPercentage",
    default_value=0.05,
)

test_split_percentage = ParameterFloat(
    name="TestSplitPercentage",
    default_value=0.05,
)

feature_store_offline_prefix = ParameterString(
    name="FeatureStoreOfflinePrefix",
    default_value="reviews-feature-store-" + str(timestamp),
)

feature_group_name = ParameterString(
    name="FeatureGroupName",
    default_value="reviews-feature-group-" + str(timestamp)
)

In [20]:
!pygmentize ./preprocess-scikit-text-to-bert-feature-store.py

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import functools
import multiprocessing

from datetime import datetime
from time import gmtime, strftime, sleep

import sys
import re
import collections
import argparse
import json
import os
import csv
import glob
from pathlib import Path
import time
import boto3
import subprocess

## PIP INSTALLS ##
subprocess.check_call([sys.executable, '-m', 'conda', 'install', '-c', 'anaconda', 'tensorflow==2.3.0', '-y'])
import tensorflow as tf
from tensorflow import keras
subprocess.check_call([sys.executable, '-m', 'conda', 'install', '-c', 'conda-forge', 'transformers==3.5.1', '-y'])
from transformers import DistilBertTokenizer
from transformers import DistilBertConfig
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'matplotlib==3.2.1'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker==2.23.1'])
import pandas as pd

import sagemaker
from sagemaker.session import Sessi

We create an instance of an `SKLearnProcessor` processor and we use that in our `ProcessingStep`.

We also specify the `framework_version` we will use throughout.

Note the `processing_instance_type` and `processing_instance_count` parameters that used by the processor instance.

In [21]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(framework_version='0.23-1',
                             role=role,
                             instance_type=processing_instance_type,
                             instance_count=processing_instance_count,
                             env={'AWS_DEFAULT_REGION': region},                             
                             max_runtime_in_seconds=7200)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [22]:
!pygmentize preprocess-scikit-text-to-bert-feature-store.py

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import functools
import multiprocessing

from datetime import datetime
from time import gmtime, strftime, sleep

import sys
import re
import collections
import argparse
import json
import os
import csv
import glob
from pathlib import Path
import time
import boto3
import subprocess

## PIP INSTALLS ##
subprocess.check_call([sys.executable, '-m', 'conda', 'install', '-c', 'anaconda', 'tensorflow==2.3.0', '-y'])
import tensorflow as tf
from tensorflow import keras
subprocess.check_call([sys.executable, '-m', 'conda', 'install', '-c', 'conda-forge', 'transformers==3.5.1', '-y'])
from transformers import DistilBertTokenizer
from transformers import DistilBertConfig
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'matplotlib==3.2.1'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker==2.23.1'])
import pandas as pd

import sagemaker
from sagemaker.session import Sessi

    print('feature_group_name {}'.format(feature_group_name))    

    # need to re-load since we can't pass feature_group object in _partial functions for some reason
    feature_group = create_or_load_feature_group(prefix, feature_group_name)
    
    filename_without_extension = Path(Path(file).stem).stem

    df = pd.read_csv(file, 
                     delimiter='\t', 
                     quoting=csv.QUOTE_NONE,
                     compression='gzip')

    df.isna().values.any()
    df = df.dropna()
    df = df.reset_index(drop=True)

    print('Shape of dataframe {}'.format(df.shape))

    if balance_dataset:  
        # Balance the dataset down to the minority class
        from sklearn.utils import resample

        five_star_df = df.query('star_rating == 5')
        four_star_df = df.query('star_rating == 4')
        three_star_df = df.query('star_rating == 3')
        two_star_df = df.query('star_rating == 2')
        one_star_df = df.query('star_rating == 1')

        mino

In [23]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processing_inputs=[
        ProcessingInput(
            input_name='raw-input-data',
            source=raw_input_data_s3_uri,
            destination='/opt/ml/processing/input/data/',
            s3_data_distribution_type='ShardedByS3Key'
        )
]

processing_outputs=[
        ProcessingOutput(output_name='bert-train',
                         s3_upload_mode='EndOfJob',
                         source='/opt/ml/processing/output/bert/train',
                        ),
        ProcessingOutput(output_name='bert-validation',
                         s3_upload_mode='EndOfJob',                         
                         source='/opt/ml/processing/output/bert/validation',
                        ),
        ProcessingOutput(output_name='bert-test',
                         s3_upload_mode='EndOfJob',
                         source='/opt/ml/processing/output/bert/test',
                        ),
]        

processing_step = ProcessingStep(
    name='Processing', 
    code='preprocess-scikit-text-to-bert-feature-store.py',
    processor=processor,
    inputs=processing_inputs,
    outputs=processing_outputs,
    job_arguments=['--train-split-percentage', str(train_split_percentage.default_value),                   
                   '--validation-split-percentage', str(validation_split_percentage.default_value),
                   '--test-split-percentage', str(test_split_percentage.default_value),
                   '--max-seq-length', str(max_seq_length.default_value),
                   '--balance-dataset', str(balance_dataset.default_value),
                   '--feature-store-offline-prefix', str(feature_store_offline_prefix.default_value),
                   '--feature-group-name', str(feature_group_name.default_value)
                  ]
)        

print(processing_step)

ProcessingStep(name='Processing', step_type=<StepTypeEnum.PROCESSING: 'Processing'>)


![Define a Processing Step for Feature Engineering](img/pipeline-2.png)

Finally, we use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to a processor instance's `run` method, for those familiar with the existing Python SDK.

Note the `input_data` parameters passed into `ProcessingStep` as the input data of the step itself. This input data will be used by the processor instance when it is run.

Also, take note the `"train_data"` and `"test_data"` named channels specified in the output configuration for the processing job. Such step `Properties` can be used in subsequent steps and will resolve to their runtime values at execution. In particular, we'll call out this usage when we define our training step.

# Train Step

In [24]:
!pygmentize src/tf_bert_reviews.py

import time
import random
import pandas as pd
from glob import glob
import pprint
import argparse
import json
import subprocess
import sys
import os
import csv
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
import tensorflow as tf
import pandas as pd
import numpy as np
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==3.5.1'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-tensorflow==2.1.0.1.0.0'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'smdebug==0.9.3'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'matplotlib==3.2.1'])

from transformers import DistilBertTokenizer
from transformers import DistilBertConfig
from transformers import TFDistilBertModel
#from transformers import TFBertForSequenceClassification

from tensorflow.keras.callbacks import ModelCheckpoint
f

                      classes=['1', '2', '3', '4', '5'], 
                      title='Confusion Matrix')

        # Save the confusion matrix        
        plt.show()
        
        # Model Output         
        metrics_path = os.path.join(local_model_dir, 'metrics/')
        os.makedirs(metrics_path, exist_ok=True)
        plt.savefig('{}/confusion_matrix.png'.format(metrics_path))
        
        report_dict = {
            "metrics": {
                "accuracy": {
                    "value": accuracy,
                },
            },
        }

        evaluation_path = "{}/evaluation.json".format(metrics_path)
        with open(evaluation_path, "w") as f:
            f.write(json.dumps(report_dict))


In [25]:
train_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.c5.9xlarge"
)

train_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=1
)

# Setup Training Hyper-Parameters
Note that `max_seq_length` is re-used from the processing hyper-parameters above

In [26]:
epochs=1
learning_rate=0.00001
epsilon=0.00000001
train_batch_size=128
validation_batch_size=128
test_batch_size=128
train_steps_per_epoch=50
validation_steps=50
test_steps=50
#train_instance_count=1
#train_instance_type='ml.c5.9xlarge'
train_volume_size=1024
use_xla=True
use_amp=True
freeze_bert_layer=False
enable_sagemaker_debugger=False
enable_checkpointing=False
enable_tensorboard=False
input_mode='File'
run_validation=True
run_test=True
run_sample_predictions=True
deploy_instance_count=1
deploy_instance_type='ml.m5.4xlarge'

# Setup Metrics To Track Model Performance

In [27]:
metrics_definitions = [
     {'Name': 'train:loss', 'Regex': 'loss: ([0-9\\.]+)'},
     {'Name': 'train:accuracy', 'Regex': 'accuracy: ([0-9\\.]+)'},
     {'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9\\.]+)'},
     {'Name': 'validation:accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'},
]

### Define a Training Step to Train a Model

We configure an Estimator and the input dataset. A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later.

We also specify the model path where the models from training will be saved.

Note the `training_instance_type` parameter passed may be also used and passed into other places in the pipeline. In this case, the `training_instance_type` is passed into the estimator.

In [28]:
# from sagemaker.utils import name_from_base
# training_job_name = name_from_base('bert-train')
# print(training_job_name)

In [29]:
# model_path = 's3://{}/{}/model'.format(default_bucket,training_job_name)
# print(model_path)

In [30]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='tf_bert_reviews.py',
                       source_dir='src',
                       role=role,
                       instance_count=train_instance_count, # Make sure you have at least this number of input files or the ShardedByS3Key distibution strategy will fail the job due to no data available
                       instance_type=train_instance_type,
                       volume_size=train_volume_size,                       
                       py_version='py37',
                       framework_version='2.3.1',
                       hyperparameters={'epochs': epochs,
                                        'learning_rate': learning_rate,
                                        'epsilon': epsilon,
                                        'train_batch_size': train_batch_size,
                                        'validation_batch_size': validation_batch_size,
                                        'test_batch_size': test_batch_size,                                             
                                        'train_steps_per_epoch': train_steps_per_epoch,
                                        'validation_steps': validation_steps,
                                        'test_steps': test_steps,
                                        'use_xla': use_xla,
                                        'use_amp': use_amp,                                             
                                        'max_seq_length': max_seq_length,
                                        'freeze_bert_layer': freeze_bert_layer,
                                        'enable_sagemaker_debugger': enable_sagemaker_debugger,
                                        'enable_checkpointing': enable_checkpointing,
                                        'enable_tensorboard': enable_tensorboard,                                        
                                        'run_validation': run_validation,
                                        'run_test': run_test,
                                        'run_sample_predictions': run_sample_predictions},
                       input_mode=input_mode,
                       metric_definitions=metrics_definitions,
#                       max_run=7200 # max 2 hours * 60 minutes seconds per hour * 60 seconds per minute
                      )

Finally, we use the estimator instance to construct a `TrainingStep` as well as the `Properties` of the prior `ProcessingStep` used as input in the `TrainingStep` inputs and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to an estimator's `fit` method, for those familiar with the existing Python SDK.

In particular, we pass in the `S3Uri` of the `"train_data"` output channel to the `TrainingStep`. We will also use the other `"test_data"` output channel for model evaluation in the pipeline. The `properties` attribute of a Workflow step match the object model of the corresponding response of a describe call. These properties can be referenced as placeholder values and are resolved, or filled in, at runtime. For example, the `ProcessingStep` `properties` attribute matches the object model of the [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response object.

In [31]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

training_step = TrainingStep(
    name='Train',
    estimator=estimator,
    inputs={
        'train': TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                'bert-train'
            ].S3Output.S3Uri,
            content_type='text/csv'
        ),
        'validation': TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                'bert-validation'
            ].S3Output.S3Uri,
            content_type='text/csv'
        ),
        'test': TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                'bert-test'
            ].S3Output.S3Uri,
            content_type='text/csv'
        )        
    },
)

print(training_step)

TrainingStep(name='Train', step_type=<StepTypeEnum.TRAINING: 'Training'>)


![Define a Training Step to Train a Model](img/pipeline-3.png)

# Evaluation Step

First, we develop an evaluation script that will be specified in a Processing step that will perform the model evaluation.

The evaluation script `evaluation.py` takes the trained model and the test dataset as input, and produces a JSON file containing classification evaluation metrics, including precision, recall, and F1 score for each label, and accuracy and ROC AUC for the model.

After pipeline execution, we will examine the resulting `evaluation.json` for analysis.

The evaluation script:

* loads in the model
* reads in the test data
* issues a bunch o' predictions against the test data
* builds a classification report, including accuracy and roc
* saves the evaluation report to the evaluation directory

Next, we create an instance of a `ScriptProcessor` processor and we use that in our `ProcessingStep`.

Note the `processing_instance_type` parameter passed into the processor.

In [32]:
from sagemaker.sklearn.processing import SKLearnProcessor

evaluation_processor = SKLearnProcessor(framework_version='0.23-1',
                                      role=role,
                                      instance_type=processing_instance_type,
                                      instance_count=processing_instance_count,
                                      env={'AWS_DEFAULT_REGION': region},
                                      max_runtime_in_seconds=7200)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [33]:
!pygmentize evaluate_model_metrics.py

import functools
import multiprocessing

from datetime import datetime
import subprocess
import sys
subprocess.check_call([sys.executable, '-m', 'conda', 'install', '-c', 'anaconda', 'tensorflow==2.3.0', '-y'])
import tensorflow as tf
from tensorflow import keras
subprocess.check_call([sys.executable, '-m', 'conda', 'install', '-c', 'conda-forge', 'transformers==3.5.1', '-y'])
from transformers import DistilBertTokenizer
from transformers import DistilBertConfig
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'matplotlib==3.2.1'])
import pandas as pd
import os
import re
import collections
import argparse
import json
import os
import numpy as np
import csv
import glob
from pathlib import Path
import tarfile
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


We use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to a processor instance's `run` method, for those familiar with the existing Python SDK.

In particular, we pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties as well as the `S3Uri` of the `"test_data"` output channel of the first `ProcessingStep`, `step_process`.

The `TrainingStep` and `ProcessingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) and  [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects, respectively.

In [34]:
from sagemaker.workflow.properties import PropertyFile

# NOTE:
# property files cause deserialization failure on listing pipeline executions
# therefore jsonget and robust conditions won't work
evaluation_report = PropertyFile(
    name='EvaluationReport',
    output_name='metrics',
    path='evaluation.json'
)

In [35]:
evaluation_step = ProcessingStep(
    name='Evaluation',
    processor=evaluation_processor,
    code='evaluate_model_metrics.py',
    inputs=[
        ProcessingInput(
            source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination='/opt/ml/processing/input/model'
        ),
        ProcessingInput(
            source=raw_input_data_s3_uri,
            #processing_step.properties.ProcessingInputConfig.Inputs['raw-input-data'].S3Input.S3Uri,
            destination='/opt/ml/processing/input/data'
        )
    ],
    outputs=[
        ProcessingOutput(output_name='metrics', 
                         s3_upload_mode='EndOfJob',
                         source='/opt/ml/processing/output/metrics/'),
    ],
    job_arguments=[
                   '--max-seq-length', str(max_seq_length.default_value),
                  ],
    property_files=[evaluation_report],  # these cause deserialization issues
)

![Define a Model Evaluation Step to Evaluate the Trained Model](img/pipeline-4.png)

In [36]:
# print(evaluation_step.properties.ProcessingOutputConfig.Outputs)

In [37]:
# from pprint import pprint
# pprint(training_step.arguments)

In [38]:
# output_config = processing_job_description['ProcessingOutputConfig']
# for output in output_config['Outputs']:
#     if output['OutputName'] == 'metrics':
#         evaluation_metrics_s3_uri = output['S3Output']['S3Uri']
        
# print(evaluation_metrics_s3_uri)

In [39]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics 

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            evaluation_step.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json"
    )
)

In [40]:
print(model_metrics)

# Register Model Step

We use the estimator instance that was used for the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a Model Package. A Model Package is a reusable model artifacts abstraction that packages all ingredients necessary for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A Model Package Group is a collection of Model Packages. You can create a Model Package Group for a specific ML business problem, and you can keep adding versions/model packages into it. Typically, we expect customers to create a ModelPackageGroup for a SageMaker Workflow Pipeline so that they can keep adding versions/model packages to the group for every Workflow Pipeline run.

The construction of `RegisterModel` is very similar to an estimator instance's `register` method, for those familiar with the existing Python SDK.

In particular, we pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

Of note, we provided a specific model package group name which we will use in the Model Registry and CI/CD work later on.

In [41]:
model_package_group_name = f"BERT-Reviews-{timestamp}"

print(model_package_group_name)

# # # NOTE: in the future, the model package group will be created automatically if it doesn't exist
# sm.create_model_package_group(
#     ModelPackageGroupName=model_package_group_name,
#     ModelPackageGroupDescription="BERT-Reviews",
# )


BERT-Reviews-1609640615


In [42]:
model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

In [43]:
inference_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.3.1",
    py_version="py37",
    instance_type="ml.m5.4xlarge",
    image_scope="inference"
)
print(inference_image_uri)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py37.


763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.1-cpu


In [44]:
from sagemaker.workflow.step_collections import RegisterModel

# NOTE: model_approval_status is not available as arg in service dsl currently
register_step = RegisterModel(
    name="RegisterModel",
    estimator=estimator,
    image_uri=inference_image_uri, # we have to specify, by default it's using training image
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.m5.4xlarge"],
    transform_instances=["ml.c5.18xlarge"],
    model_package_group_name=model_package_group_name,
# Note:  this isn't being picked up, so we're commenting it out
#    model_approval_status=model_approval_status  
    model_metrics=model_metrics
    
)

## Define a Condition Step to Check Accuracy and Conditionally Register Model

Finally, we'd like to only register this model if the accuracy of the model, as determined by our evaluation step `step_eval`, exceeded some value. A `ConditionStep` allows for pipelines to support conditional execution in the pipeline DAG based on conditions of step properties. 

Below, we:

* define a `ConditionGreaterThan` on the accuracy value found in the output of the evaluation step, `step_eval`.
* use the condition in the list of conditions in a `ConditionStep`
* pass the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`
* use the `FailStep` in the `else_steps` of the `ConditionStep` to fail the pipeline if the accuracy condition was not met

NOTE: there are a few things that are planned to be implemented in the Workflow service that are currently unavailable to us:

* JsonGet - a function to allow getting json files from S3 and using their values in conditions
* FailStep - a step that terminates the pipeline in failure

In [45]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)

minimum_accuracy_condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step=evaluation_step,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value",
    ),
    right=0 # accuracy percent
)

minimum_accuracy_condition_step = ConditionStep(
    name="AccuracyCondition",
    conditions=[minimum_accuracy_condition],
    if_steps=[register_step], # success, continue with model registration
    else_steps=[], # fail, end the pipeline
)

![Define a Condition Step to Check Accuracy and Conditionally Register Model](img/pipeline-5.png)

## Define a Pipeline of Parameters, Steps, and Conditions

Let's tie it all up into a workflow pipeline so we can execute it, and even schedule it.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair so we tack on the timestamp to the name.

Note:

* All the parameters used in the definitions must be present.
* Steps passed into the pipeline need not be in the order of execution. The SageMaker Workflow service will resolve the _data dependency_ DAG as steps the execution complete.
* Steps must be unique to either pipeline step list or a single condition step if/else list.

![Define a Pipeline of Parameters, Steps, and Conditions](img/pipeline-6.png)

In [46]:
from sagemaker.workflow.pipeline import Pipeline

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        input_data,
        processing_instance_count,
        processing_instance_type,
        max_seq_length,
        balance_dataset,
        train_split_percentage,
        validation_split_percentage,
        test_split_percentage,
        feature_store_offline_prefix,
        feature_group_name,        
        train_instance_type,
        train_instance_count,
        model_approval_status,
        deploy_instance_type,
        deploy_instance_count        
    ],
    steps=[processing_step, training_step, evaluation_step, minimum_accuracy_condition_step], # register_step],
    sagemaker_session=sess,
)

Let's examine the Json of the pipeline definition that meets the SageMaker Workflow Pipeline DSL specification.

By examining the definition, we're also confirming that the pipeline was well-defined, and that the parameters and step properties resolve correctly.

In [47]:
import json
from pprint import pprint

definition = json.loads(pipeline.definition())
pprint(definition)

{'Metadata': {},
 'Parameters': [{'DefaultValue': 's3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/',
                 'Name': 'InputData',
                 'Type': 'String'},
                {'DefaultValue': 1,
                 'Name': 'ProcessingInstanceCount',
                 'Type': 'Integer'},
                {'DefaultValue': 'ml.c5.2xlarge',
                 'Name': 'ProcessingInstanceType',
                 'Type': 'String'},
                {'DefaultValue': 64, 'Name': 'MaxSeqLength', 'Type': 'Integer'},
                {'DefaultValue': 'True',
                 'Name': 'BalanceDataset',
                 'Type': 'String'},
                {'DefaultValue': 0.9,
                 'Name': 'TrainSplitPercentage',
                 'Type': 'Float'},
                {'DefaultValue': 0.05,
                 'Name': 'ValidationSplitPercentage',
                 'Type': 'Float'},
                {'DefaultValue': 0.05,
                 'Name': 'TestSplitPercentage',
           

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [48]:
print(experiment_name)

BERT-pipeline-16096406156145956


## Ignore the `WARNING` below

In [49]:
response = pipeline.create(role_arn=role
#                           experiment_name=experiment_name # Experiments not natively supported by pipeline
                          )

pipeline_arn = response["PipelineArn"]
print(pipeline_arn)

arn:aws:sagemaker:us-east-1:835319576252:pipeline/bert-pipeline-16096406156145956


We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [50]:
execution = pipeline.start(
# TODO:  ADD THE PARAMETERS - SAME AS 2nd RUN BELOW
#         InputDataUrl=raw_input_data_s3_uri,
#         ProcessingInstanceCount=1,
#         ProcessingInstanceType='ml.c5.2xlarge',
#         MaxSeqLength=64,
#         BalanceDataset='True',
#         TrainSplitPercentage=0.9,
#         ValidationSplitPercentage=0.05,
#         TestSplitPercentage=0.05,
# #        FeatureStoreOfflinePrefix='', # TODO:  pass this in or leave empty for default?
# #        FeatureGroupName='', # TODO:  pass this in or leave empty for default?
#         TrainingInstanceType='ml.c5.9xlarge',
#         TrainingInstanceCount=1,
#         ModelApprovalStatus="Approved", # TODO:  Figure out why this is not working
#         DeployInstanceType='ml.m5.4xlarge',
#         DeployInstanceCount=1    
)

print(execution.arn)

arn:aws:sagemaker:us-east-1:835319576252:pipeline/bert-pipeline-16096406156145956/execution/kdcvcnkx1smp


### Workflow Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [51]:
from pprint import pprint

execution_run = execution.describe()
pprint(execution_run)

{'CreatedBy': {},
 'CreationTime': datetime.datetime(2021, 1, 3, 2, 23, 43, 321000, tzinfo=tzlocal()),
 'LastModifiedBy': {},
 'LastModifiedTime': datetime.datetime(2021, 1, 3, 2, 23, 43, 321000, tzinfo=tzlocal()),
 'PipelineArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/bert-pipeline-16096406156145956',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/bert-pipeline-16096406156145956/execution/kdcvcnkx1smp',
 'PipelineExecutionDisplayName': 'execution-1609640623425',
 'PipelineExecutionStatus': 'Executing',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '427',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Sun, 03 Jan 2021 02:23:42 GMT',
                                      'x-amzn-requestid': 'd0a9dadf-7a96-4105-ad01-5b9817d8a8ab'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd0a9dadf-7a96-4105-ad01-5b9817d8a8ab',
     

## Add Execution Run as Trial to Experiments

In [52]:
execution_run_name = execution_run['PipelineExecutionDisplayName']
print(execution_run_name)

execution-1609640623425


# List Execution Steps

In [53]:
import time

# Giving the first step time to start up
time.sleep(30)

execution.list_steps()

[{'StepName': 'Processing',
  'StartTime': datetime.datetime(2021, 1, 3, 2, 23, 44, 8000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:835319576252:processing-job/pipelines-kdcvcnkx1smp-processing-xbtw8fwfaf'}}}]

We can wait for the execution by invoking `wait()` on the execution:

In [ ]:
%%time

execution.wait()

We can list the execution steps to check out the status and artifacts:

In [ ]:
execution.list_steps()

# Examining the Evalution

Examine the resulting model evaluation after the pipeline completes. Download the resulting evaluation.json file from S3 and print the report.




In [ ]:
print(evaluation_step.properties.ProcessingOutputConfig.Outputs['metrics'].S3Output.S3Uri)


# arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]

# .__dict__ #  .ModelArtifacts.S3ModelArtifacts

In [ ]:
from pprint import pprint

print(evaluation_metrics_s3_uri)

# TODO:  Change this to ProcessingOutput when we switch to PropertyFile per https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html
evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
    evaluation_metrics_s3_uri
))

pprint(json.loads(evaluation_json))

In [ ]:
# from sagemaker.workflow.properties import PropertyFile

# evaluation_report = PropertyFile(
#     name="EvaluationReport",
#     output_name="evaluation",
#     path="evaluation.json"
# )

## List All Artifacts Generated By The Pipeline

In [ ]:
processing_job_name=None
training_job_name=None

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step['StepName'] == 'Processing':
        processing_job_name=execution_step['Metadata']['ProcessingJob']['Arn'].split('/')[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step['StepName'] == 'Train':
        training_job_name=execution_step['Metadata']['TrainingJob']['Arn'].split('/')[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)


## Add Trial Compontents To Experiment Trial

In [ ]:
print(processing_job_name)

In [ ]:
processing_job_tc = '{}-aws-processing-job'.format(processing_job_name)
print(processing_job_tc)

In [ ]:
response = sm.associate_trial_component(
    # -aws-processing-job is the default name assigned by ProcessingJob
    TrialComponentName=processing_job_tc,
    TrialName=trial_name
)

In [ ]:
print(training_job_name)

In [ ]:
training_job_tc = '{}-aws-training-job'.format(training_job_name)
print(training_job_tc)

In [ ]:
response = sm.associate_trial_component(
    # -aws-training-job is the default name assigned by TrainingJob
    TrialComponentName=training_job_tc,
    TrialName=trial_name
)

## Log Additional Parameters within Trial

In [ ]:
from smexperiments import tracker

processing_job_tracker = tracker.Tracker.load(trial_component_name=processing_job_tc)
print(processing_job_tracker)

In [ ]:
processing_job_tracker.log_parameters({
    "balance_dataset": str(balance_dataset), 
})

# must save after logging
processing_job_tracker.trial_component.save()

### Examining the evalution

We can examine the resulting model evaluation after the pipeline completes.

We download the resulting `evaluation.json` file from S3 and print the report.

In [ ]:
# sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
#     step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
# ))

## ------ Run another Pipeline Execution with Different Parameters ------

In [ ]:
###
# THIS IS RUNNING A 2nd PIPELINE
###
execution_parametrized = pipeline.start(
    parameters=dict(
#         InputData='',
#         ProcessingInstanceCount=1,
#         ProcessingInstanceType='',
#         MaxSeqLength=64,
#         TrainSplitPercentage=0.9,
#         ValidationSplitPercentage=0.05,
#         TestSplitPercentage=0.05,
#         TrainingInstanceType='',
#         TrainingInstanceCount=1,
        ModelApprovalStatus='Approved'
    )
)

# Analyze Experiment

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

import pandas as pd
pd.set_option("max_colwidth", 500)
#pd.set_option("max_rows", 100)

experiment_analytics = ExperimentAnalytics(
    experiment_name=experiment_name,
)

experiment_analytics_df = experiment_analytics.dataframe()
experiment_analytics_df

# List All Artifacts

In [ ]:
# from sagemaker.analytics import ArtifactAnalytics
# analytics = ArtifactAnalytics()
# artifact_analytics_df = analytics.dataframe()
# artifact_analytics_df

## The Model Registry and CI/CD


The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

Let's check it out:

In [ ]:
for execution_step in execution.list_steps():
    if execution_step["StepName"] == "RegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break
print(model_package_arn)

## Update Model Package Approval Status

As noted above, the model has been registered with `"PendingManualApproval"` status. As part of Yosemite, data scientists can register the model with approved/pending manual approval as part of the Tioga workflow. Here we are demonstrating how they can approve the generated model manually. In GA (Nov 2020), we will have UX in SageMaker Studio so that datascients can approve the model, which will inturn trigger the Ci/CD system. For now, here is a way to approve it.

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",
)

print(model_package_update_response)

# Deploy Model

In [ ]:
import time
timestamp = int(time.time())

## Create Model From Model Registry

In [ ]:
# model_name = 'bert-model-{}'.format(timestamp)
# print("Model name : {}".format(model_name))
# primary_container = {
#     'ModelPackageName': model_package_arn,
# }
# create_model_response = sm.create_model(
#     ModelName = model_name,
#     ExecutionRoleArn = role,
#     PrimaryContainer = primary_container
# )
# print("Model arn : {}".format(create_model_response["ModelArn"]))

In [ ]:
# from sagemaker.model import Model

# model = Model(
#     image_uri=image_uri,
#     model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
#     sagemaker_session=sagemaker_session,
#     role=role,
# )

In [ ]:
# from sagemaker.inputs import CreateModelInput
# from sagemaker.workflow.steps import CreateModelStep

# inputs = CreateModelInput(
#     instance_type="ml.m5.large",
#     accelerator_type="ml.eia1.medium",
# )
# step_create_model = CreateModelStep(
#     name="AbaloneCreateModel",
#     model=model,
#     inputs=inputs,
# )


## Create Endpoint Config

In [ ]:
endpoint_config_name = 'bert-model-epc-{}'.format(timestamp)
print(endpoint_config_name)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m5.4xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

## Create Endpoint

In [ ]:
endpoint_name = 'bert-model-ep-{}'.format(timestamp)
print("EndpointName={}".format(endpoint_name))

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, endpoint_name)))


# _Wait Until the Endpoint is Deployed_

In [ ]:
%%time

waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

# Test the Deployed Model

In [ ]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name=endpoint_name,
                                sagemaker_session=sess,
                                model_name='saved_model',
                                model_version=0)

# Predict the `star_rating` with Ad Hoc `review_body` Samples

In [ ]:
reviews = ["This is great!"]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)